In [1]:
import numpy as np
import sys
import numpy
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# ------- Functions to help calculate resolution of cameras ---------------------

def find_change(arr, value):  
    # Finds indices of array where the pixel value between index i and i+2 is greater than the specified value.
    # Input:
        # arr: array of pixel values of row or column. Ex: arr = imarray[100,:].
        # value: int. Amount by which the pixel value at index i+2 must differ from pixel value at i to be 
        #             counted as an edge. 
    # Returns:
        # indices: list of indices.
    indices = []
    for idx, x in np.ndenumerate(arr):
        for i in idx:
            if i < len(arr)-2:
                if abs(int(arr[i]) - int(arr[i + 2])) > value:
                    indices.append(i)
    return indices

def split_nonconsec(arr):  
    # Splits an array into multiple arrays at non-consecutive values.
    # Input:
        # arr: array.
    # Returns:
        # split_arrs: list of arrays of consecutive values.
    non_consec= []
    for i in range(len(arr)-1):
        if arr[i+1] != arr[i]+1:
            non_consec.append(i+1)
    split_arrs = np.split(arr, non_consec)
    return split_arrs

def midpoints(split_arrs):  
    #Finds midpoint of an array of consecutive values and rounds it to nearest integer.
    # Input:
        # split_arrs: list of arrays. 
    # Returns:
        # mids: list of midpoints.
    mids = []
    for arr in split_arrs:
        mids.append(round(sum(arr)/len(arr)))
    return mids

def find_edges(arr, value):
    # Finds the edges where the pixel value changes more than the specified value by finding the midpoint 
    # of each group of consecutive changing indices.
    # Input: 
        # arr: array of pixel values of the row or column being analyzed. Ex: arr = imarray[100,:].
        # value: int. Amount by which the pixel value at index i+2 must differ from pixel value at i to be 
        #             counted as an edge. 
    # Returns:
        # edges: list of index values that define each edge. 
    change = find_change(arr, value)
    split_arrs = split_nonconsec(change)
    edges = midpoints(split_arrs)
    return edges

def find_incr(arr, edges):
    # Finds the values of the edges with increasing slope
    # Input: 
        # arr: array of pixel values of the row or column being analyzed. Ex: arr = imarray[100,:].
        # edges: array of edge values (output of find_edges function)
    # Returns: 
        # incr: list of index values of edges with increasing slope
    incr = []
    for edg in edges:
        if int(arr[int(edg)])> int(arr[int(edg-2)]):
            incr.append(int(edg))
    return incr

def find_decr(arr, edges):
    # Finds the values of the edges with decreasing slope
    # Input: 
        # arr: array of pixel values of the row or column being analyzed.
        # edges: array of edge values (output of find_edges function)
    # Returns: 
        # decr: list of index values of edges with decreasing slope
    decr = []
    for edg in edges:
        if int(arr[int(edg)]) < int(arr[int(edg-2)]):
            decr.append(int(edg))
    return decr

def horiz_lines(xmin, xmax, ymin, ymax, lppmm, value=5000):
    # Calculates the average horizontal line pair width and the average pixels per mm for a specific group
    # and element. Image array must be named imarray.
    # Input:
        # xmin, xmax: int. Minimum and maximum x values of image segment. For horizontal lines, xmin and xmax
        #             should be within the edges of the black lines
        # ymin, ymax: int. Minumum and maximum y values of image segment. For horizontal lines, ymin and ymax
        #             should be well outside the edges of the black lines. 
        # lppmm: int. Number of line pairs per mm for the group and element being analyzed. Found in online
        #             specs for specific Air Force Target. 
        # value: int. Amount by which the pixel values must differ to be counted as an edge. Default is 5000.
    # Output: Prints the calculated average pixels per mm and average pixel width in um. 

    segment = imarray[ymin:ymax,xmin:xmax]
    wid = segment.shape[1]
#    plt.imshow(segment)   # Displays image segment
    width = []  # Array for width of horizontal lines
    for i in np.arange(0,wid,1):
        row = segment[:,i]
        edg = find_edges(row, value)
        incr = find_incr(row, edg)
        decr = find_decr(row, edg)
        for i in range(len(decr)-1):
            width.append(decr[i+1]-decr[i])
        for j in range(len(incr)-1):
            width.append(incr[j+1]-incr[j])
    avg_width = round(sum(width)/len(width),2)
#    print('Avg. horiz. line pair width = ' + str(avg_width) + ' pixels/line pair')
    pixels_mm = avg_width * lppmm
    pixel_width = 1/pixels_mm
    print('Avg. pixels/mm: ' + str(round(pixels_mm,2)))
    print('Avg. pixel width: ' + str(round(pixel_width*10**3,2)) + ' \u03BC'+ 'm')

def vert_lines(xmin, xmax, ymin, ymax, lppmm, value=5000):
    # Calculates the average vertical line pair width and the average pixels per mm for a specific group
    # and element. Image array must be named imarray. 
    # Input:
        # xmin, xmax: int. Minimum and maximum x values of image segment. For vertical lines, xmin and xmax
        #             should be well outside the edges of the black lines
        # ymin, ymax: int. Minumum and maximum y values of image segment. For vertical lines, ymin and ymax
        #             should be within the edges of the black lines. 
        # lppmm: int. Number of line pairs per mm for the group and element being analyzed. Found in online
        #             specs for specific Air Force Target. 
        # value: int. Amount by which the pixel values must differ to be counted as an edge. Default is 5000.
    # Output: Prints the calculated average pixels per mm and average pixel width in um. 

    segment = imarray[ymin:ymax,xmin:xmax]
    height = segment.shape[0]
#    plt.imshow(segment)  # Displays image segment
    width = []  # Array for width of horizontal lines
    for i in np.arange(0,height,1):
        row = segment[i,:]
        edg = find_edges(row, value)
        incr = find_incr(row, edg)
        decr = find_decr(row, edg)
        for i in range(len(decr)-1):
            width.append(decr[i+1]-decr[i])
        for j in range(len(incr)-1):
            width.append(incr[j+1]-incr[j])
    avg_width = round(sum(width)/len(width),2)
#    print('Avg. vert. line pair width = ' + str(avg_width) + ' pixels/line pair')
    pixels_mm = avg_width * lppmm
    pixel_width = 1/pixels_mm
    print('Avg. pixels/mm: ' + str(round(pixels_mm,2)))
    print('Avg. pixel width: ' + str(round(pixel_width*10**3,2)) + ' \u03BC'+'m')

def avg_ratio_horiz(xmin, xmax, ymin, ymax):
    # Finds average ratio of min pixel value to max pixel value per column in a set of horizontal lines. 
    # Input: 
        # xmin, xmax: int. Min and max x values of image segment. For horizontal lines, xmin and xmax
        #             should be inside the edges of the black lines
        # ymin, ymax: int. Minumum and maximum y values of image segment. For horizontal lines, ymin and ymax
        #             should be well outside the edges of the black lines.
    # Returns: 
        # avg_ratio: float64. 
    
    segment = imarray[ymin:ymax,xmin:xmax]
    wid = segment.shape[1]
    ratios = []
    for i in np.arange(0,wid,1):
        col = segment[:,i]
        min_val = np.amin(col)
        max_val = np.amax(col)
        ratios.append(min_val/max_val)
    avg_ratio = round(sum(ratios)/len(ratios),4)
    return avg_ratio

def avg_ratio_vert(xmin, xmax, ymin, ymax):
    # Finds average ratio of min pixel value to max pixel value per column in a set of horizontal lines. 
    # Input: 
        # xmin, xmax: int. Min and max x values of image segment. For vertical lines, xmin and xmax
        #             should be well outside the edges of the black lines
        # ymin, ymax: int. Min and max y values of image segment. For vertical lines, ymin and ymax
        #             should be inside the edges of the black lines.
    # Returns: 
        # avg_ratio: float64. 
        
    segment = imarray[ymin:ymax,xmin:xmax]
    height = segment.shape[0]
    ratios = []
    for i in np.arange(0,height,1):
        row = segment[i,:]
        min_val = np.amin(row)
        max_val = np.amax(row)
        ratios.append(min_val/max_val)
    avg_ratio = round(sum(ratios)/len(ratios),4)
    return avg_ratio

#----------- Extra functions that might be useful ------------------

def find_midpoints(edges):
    # Finds the midpoints between the edges. 
    # Input: 
        # edges: array of edge values (output of find_edges function)
    # Returns:
        # midpoints: list of midpoint values. 
    midpoints = []
    for i in range(len(edges)-1):
        midpoints.append((edges[i] + (edges[i+1]))/2)
    return midpoints

def print_edge_values(arr, edges):
    # Prints a table of the indices and the pixel values at each edge
    # Input:
        # arr: array of pixel values of the row or column being analyzed.
        # edges: array of edge values (output of find_edges function)
    print('Edges \nIndex  Value')
    for edge in edges:
        print(str(edge) + ': '+ str(arr[int(edge)]))

def print_mid_values(arr, midpoints):
    #Prints a table of the midpoints' indices, pixel values, and relative color (light or dark)
    # Input:
        # arr: array of pixel values of the row or column being analyzed.
        # midpoints: array of midpoint values (output of find_midpoints function)
    print('Midpoints\nIndex     Value     Color')
    if arr[int(midpoints[0])-int(midpoints[0]/2)]<arr[int(midpoints[0])]:
            print(str(midpoints[0]-int(midpoints[0]/2))+ '      ' + str(arr[int(midpoints[0]-int(midpoints[0]/2))]) + '    dark')
    elif arr[int(midpoints[0])-int(midpoints[0]/2)]>arr[int(midpoints[0])]:
            print(str(midpoints[0]-int(midpoints[0]/2))+ '      ' + str(arr[int(midpoints[0]-int(midpoints[0]/2))]) + '    light')

    for i in range(len(midpoints)-1):
        if i==0:
            if arr[int(midpoints[i])]<arr[int(midpoints[i+1])]:
                print(str(midpoints[i])+ '      ' + str(arr[int(midpoints[i])]) + '    dark')
            elif arr[int(midpoints[i])]>arr[int(midpoints[i+1])]:
                print(str(midpoints[i])+ '      ' + str(arr[int(midpoints[i])]) + '   light')
        elif i==len(midpoints)-1:
            if arr[int(midpoints[i])]<arr[int(midpoints[i-1])]:
                print(str(midpoints[i])+ '      ' + str(arr[int(midpoints[i])]) + '    dark')
            elif arr[int(midpoints[i])]>arr[int(midpoints[i-1])]:
                print(str(midpoints[i])+ '      ' + str(arr[int(midpoints[i])]) + '   light')
        else:
            if arr[int(midpoints[i])]<arr[int(midpoints[i+1])] and arr[int(midpoints[i])]<arr[int(midpoints[i-1])]:
                print(str(midpoints[i])+ '      ' + str(arr[int(midpoints[i])]) + '    dark')
            elif arr[int(midpoints[i])]>arr[int(midpoints[i+1])] and arr[int(midpoints[i])]>arr[int(midpoints[i-1])]:
                print(str(midpoints[i])+ '      ' + str(arr[int(midpoints[i])]) + '   light')

In [3]:
im = Image.open('image_1.tiff')
imarray = numpy.array(im)
%matplotlib tk
plt.imshow(imarray)   #Plot the image in a popout window

In [41]:
# Analyzing Group 3, Element 6, horizontal lines
# Horizontal lines extend from x = 602 to 620 and y = 870 to 900 
# Number of line pairs per mm is 14.30
horiz_lines(602,620,870,900,14.30,2000)

Avg. pixels/mm: 137.85
Avg. pixel width: 7.25 μm


In [42]:
# Analyzing Group 2, Element 6, vertical lines
# Vertical lines extend from x = 885 to 950 and y = 731 to 771 
# Number of line pairs per mm is 7.13
vert_lines(885,950,731,771,7.13,5000)

Avg. pixels/mm: 133.69
Avg. pixel width: 7.48 μm


In [43]:
# Analyzing Group 2, Element 6, horizontal lines
# Horiz lines extend from x = 963 to 1003 and y = 720 to 780
# Number of line pairs per mm is 7.13
horiz_lines(963,1003,720,780,7.13,5000)

Avg. pixels/mm: 135.4
Avg. pixel width: 7.39 μm


In [53]:
# Analyzing Group 0, Element 1, horizontal lines
# Horiz lines extend from x = 188 to 385 and y = 20 to y = 400
# Number of line pairs per mm is 1.00
horiz_lines(188, 385, 20, 400, 1.00, 8000)

Avg. pixels/mm: 137.48
Avg. pixel width: 7.27 μm


In [52]:
# Analyzing Group 0, Element 1, vertical lines
# Vertical lines extend from x = 647 to 1018 and y = 53 to y = 386
# Number of line pairs per mm is 1.00
vert_lines(647, 1018, 53, 386, 1.00, 8000)

Avg. pixels/mm: 137.36
Avg. pixel width: 7.28 μm


In [46]:
# Analyzing Group 0, Element 6, vertical lines
# Vertical lines extend from x = 1351 to 1564 and y = 58 to y = 239
# Number of line pairs per mm is 1.00
vert_lines(1351, 1564, 58, 239, 1.78, 8000)

Avg. pixels/mm: 135.0
Avg. pixel width: 7.41 μm


In [85]:
# Calculating average ratio of min to max pixel values to find which group and element has ratio closest to 0.5.

# Group 4, Element 6:
vert46 = avg_ratio_vert(785,796,889,898)
horiz46 = avg_ratio_horiz(803,812,889,900)
print('4/6: ' + str((vert46+horiz46)/2))

# Group 4, Element 5:
vert45 = avg_ratio_vert(781,793,906,918)
horiz45 = avg_ratio_horiz(801,812,905,919)
print('4/5: ' + str((vert45+horiz45)/2))

# Group 4, Element 2:
vert42 = avg_ratio_vert(768,786,971,988)
horiz42 = avg_ratio_horiz(794,812,969,988)
print('4/2: ' + str((vert42+horiz42)/2))

# Group 3, Element 1:
vert31 = avg_ratio_vert(659,700,1094,1136)
horiz31 = avg_ratio_horiz(600,640,1094,1136)
print('3/1: ' + str((vert31+horiz31)/2))

# Group 3, Element 4:
vert34 = avg_ratio_vert(643,668,948,974)
horiz34 = avg_ratio_horiz(600,628,947,974)
print('3/4: ' + str((vert34+horiz34)/2))

# Group 3, Element 6:
vert36 = avg_ratio_vert(633,655,875,896)
horiz36 = avg_ratio_horiz(602,621,875,896)
print('3/6: ' + str((vert36+horiz36)/2))

# Group 4, Element 1:
vert41 = avg_ratio_vert(743,762,890,909)
horiz41 = avg_ratio_horiz(714,733,889,909)
print('4/1: ' + str((vert41+horiz41)/2))

# Group 4, Element 2:
vert42 = avg_ratio_vert(769,785,971,986)
horiz42 = avg_ratio_horiz(796,810,971,987)
print('4/2: ' + str((vert42+horiz42)/2))

# Group 5, Element 4:
vert54 = avg_ratio_vert(724,731,943,949)
horiz54 = avg_ratio_horiz(714,720,943,950)
print('5/4: ' + str((vert54+horiz54)/2))

# Group 5, Element 3:
vert53 = avg_ratio_vert(726,733,954,961)
horiz53 = avg_ratio_horiz(714,721,953,961)
print('5/3: ' + str((vert53+horiz53)/2))

4/6: 0.69705
4/5: 0.66635
4/2: 0.5809
3/1: 0.29485
3/4: 0.3594
3/6: 0.42355
4/1: 0.48945
4/2: 0.5692999999999999
5/4: 0.81705
5/3: 0.7987


In [11]:
# Extra commands:

#numpy.set_printoptions(threshold=sys.maxsize)  # Display entire array

#im.size  # (width, height)
#imarray.shape #(height, width)
#indexing a 2D array: imarry[row][column]

#min_value = np.amin(test)
#print('Min value = ' + str(min_value))

#max_value = np.amax(test)
#print('Max value = ' + str(max_value))

#avg_value = np.average(test)
#print('Avg value = ' + str(avg_value))

#min_value_index = np.argmin(test)
#print(min_value_index)

#row100 = imarray[100,:]
#ed = find_edges(row100, 5000)
#mid = find_midpoints(ed)
#print_edge_values(row100, ed)
#print_mid_values(row100, mid)
#print(ed)
#print(find_incr(row100, ed))
#print(find_decr(row100, ed))

# -- Plot a lineout ----
row100 = imarray[100,:]
plt.plot(row100, label="Row 100")
#plt.scatter(range(len(row100)),row100)
#plt.legend(loc = 'upper right')
plt.xlabel('Pixel number')
plt.ylabel('Pixel value')
plt.show()